# Notebook for Topic 5 - Time Series anomaly detection

<hr style="border-top: 1px solid #001a79;" />

## Introduction

This Notebook is a recreation of instructions from the blogpost 'Timeseries anomaly detection using an Autoencoder' available https://keras.io/examples/timeseries/timeseries_anomaly_detection/

In this notebook, I will demonstrate the use of Keras and TensorFlow to build and train a model for timeseries anomaly detection using the NAB (Numenta Anomaly Benchmark) dataset. The NAB dataset is a collection of labeled timeseries data files that are commonly used to evaluate the performance of anomaly detection algorithms. I will use Keras and TensorFlow to build and train a deep learning model that can detect anomalies in the NAB timeseries data. I will start by setting up the environment and loading the necessary libraries, then I will explore the NAB dataset and preprocess the data for use with the model. Next, I will build and train the model, and finally I will evaluate its performance on the NAB dataset.

In [ ]:
# Import the necessary modules


from pprint import pprint

# library for numerical computing
import numpy as np

# library for data manipulation and analysis
import pandas as pd

# package for building and training deep learning models in TensorFlow.
from tensorflow import keras
from tensorflow.keras import layers

# library for data visualization
from matplotlib import pyplot as plt

## Download the data

In this example, data from the NAB Data Corpus will be used: https://github.com/numenta/NAB/tree/master/data

The NAB corpus is a collection of 58 timeseries data files that are intended for use in research on streaming anomaly detection. The data includes both real-world and artificial timeseries data, and contains labeled anomalous periods of behavior. The data is composed of ordered, timestamped, single-valued metrics.

In [ ]:
# URL for the Github NAB data repository
master_url_root = "https://raw.githubusercontent.com/numenta/NAB/master/data/"

# Parse a URL for dataset without anomaly
df_small_noise_url_suffix = "artificialNoAnomaly/art_daily_small_noise.csv"
df_small_noise_url = master_url_root + df_small_noise_url_suffix

# Parse a URL for dataset with anomaly
df_daily_jumpsup_url_suffix = "artificialWithAnomaly/art_daily_jumpsup.csv"
df_daily_jumpsup_url = master_url_root + df_daily_jumpsup_url_suffix



# Download dataset without anomaly
df_small_noise = pd.read_csv(
    df_small_noise_url, parse_dates=True, index_col="timestamp"
)

# Download dataset with anomaly
df_daily_jumpsup = pd.read_csv(
    df_daily_jumpsup_url, parse_dates=True, index_col="timestamp"
)

#### Print the 5 first rows of data

Display the first few rows of downloaded data for previewing, checking the integrity of, and debugging the data.

In [ ]:
print("First 5 rows of data without Anomalies:")
df_small_noise.head()

In [ ]:
print("First 5 rows of data with Anomalies:")
df_daily_jumpsup.head()

## Visualise the data

#### Data without anomalies

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))
df_small_noise.plot(legend=False, ax=ax)
plt.show()

#### Data with anomalies

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))
df_daily_jumpsup.plot(legend=False, ax=ax)
plt.show()

## Prepare the training data

For training purposes, the dataset without anomalies will bu used.

#### Normalize the data

Normalization is a common preprocessing step in machine learning that involves transforming the data so that it has zero mean and unit variance. This can be useful for algorithms that are sensitive to the scale of the input data, as it helps to ensure that all features are on a similar scale

In [ ]:
# Calculate and save the values of the mean and standard deviation of the no anomalies dataset
training_mean = df_small_noise.mean()
training_std = df_small_noise.std()

# Displaye the values
print('Mean value of no anomalies dataset: {:.3f}'.format(training_mean.value))
print('Standard Deviation of no anomalies dataset: {:.3f}'.format(training_std.value))

# Normalize the no anomalies dataset
df_training_value = (df_small_noise - training_mean) / training_std
print("\nNumber of training samples:", len(df_training_value))

#### Visualise the normalized no anomalies dataset

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))
df_training_value.plot(legend=False, ax=ax)
plt.show()

#### Create sequences

The function below will be used to create a set of input sequences for training a ML model. This function will create overlapping sequences of the input values, with each sequence shifted by 1 value comparing to the previous one. This allows the model to learn dependencies between the values at different time steps.

In [ ]:
TIME_STEPS = 288

# Generated training sequences for use in the model.
def create_sequences(values, time_steps=TIME_STEPS, verbose=False):
    if verbose:
        print("Input Array:")
        print(values)
        print("\nSliding Windows:")        
        
    output = []
    for i in range(len(values) - time_steps + 1):
        app_values = values[i : (i + time_steps)]
        if verbose:
            print("Window nr {}: {}".format(i+1, app_values))
        output.append(app_values)
        
    if verbose:
        print("\nFinal Array:")
        pprint(output)
        
    return np.stack(output)

In [ ]:
# Example of 'create_sequences' usage:
seq_example = create_sequences([1,2,3,4,5,6,7,8,9,10], time_steps=4, verbose=True);
seq_example.shape

In [ ]:
# The alternative below should be more efficient than the original implementation of 'create_sequences',
# as it avoids the overhead of creating and appending to a Python list and use more efficient NumPy array slicing instead.

def create_sequences(values, time_steps=TIME_STEPS):
    values = np.asarray(values)
    return np.stack([values[i : (i + time_steps)] for i in range(len(values) - time_steps + 1)])

In [ ]:
# Use normalised no anomalies dataset

x_train = create_sequences(df_training_value.values)

In [ ]:
# Display the shape of the training input data x_train 

print("\nShape of the the training input data x_train:\n")
print(" Number of sequences: {}\n Sequence length: {} \n Sequence Height: {}".format(x_train.shape[0], x_train.shape[1], x_train.shape[2]))

## Build a Model

#### What is Convolution?

Convolution is a mathematical operation that is commonly used in signal and image processing to modify or extract information from a signal or image. It involves sliding a kernel (also called a filter or mask) over the input matrix and applying element-wise multiplication and summing the results to produce a single output value at each position.

Below is the example of convolution of the 1D input array A = [1,2,3,4,5] and the kernel = [1,2,3]. Note below, that in the convolution operation, the second array (kernel) is flipped.

$C_1 = 1*1 = 1$

|              |    |    |    |    |    |    |    |    |    |     $C_1$   |
|--------------|----|----|----|----|----|----|----|----|----|--------------|
| Input Array  |    |    | <td style="border: 1px solid red;">**1** </td> |  2  | 3  | 4  | 5  |   |    |  **1**      |
| Kernel       | 3  | 2  | <td style="border: 1px solid red;">**1** </td>      |    |    |    |      |    |           |


$C_2 = 1*2+2*1 = 4$

|              |    |    |    |    |    |    |    |    |    |     $C_2$   |
|--------------|----|----|----|----|----|----|----|----|----|--------------|
| Input Array  |    |    |<td style="border: 1px solid red;">**1** </td>   <td style="border: 1px solid red;">**2** </td> |  3  | 4  | 5  |    |    |   **4**      |
| Kernel       |    | 3  |<td style="border: 1px solid red;">**2** </td>   <td style="border: 1px solid red;">**1** </td>       |    |    |       |    |              |

$C_3 = 1*3+2*2+3*1 = 10$

|              |    |    |    |    |    |    |    |    |    |     $C_3$   |
|--------------|----|----|----|----|----|----|----|----|----|--------------|
| Input Array  |    |    |<td style="border: 1px solid red;">**1** </td>   <td style="border: 1px solid red;">**2** </td><td style="border: 1px solid red;">**3** </td> | 4 |5 | | |**10** |
| Kernel       |    |    | <td style="border: 1px solid red;">**3** </td>  <td style="border: 1px solid red;">**2** </td>   <td style="border: 1px solid red;">**1** </td>  | | | | |


$C_4 = 2*3 + 3*2 + 4*1 = 16$

|            |    |    |    |    |    |    |    |    |      |  $C_4$   |
|--------------|----|----|----|----|----|----|----|----|------|----------|
| Input Array  |    |    | 1  <td style="border: 1px solid red;">**2** </td><td style="border: 1px solid red;">**3** </td>  <td style="border: 1px solid red;">**4** </td> |5 | | | |**16** |
| Kernel       |    |    | <td style="border: 1px solid red;">**3** </td>  <td style="border: 1px solid red;">**2** </td>   <td style="border: 1px solid red;">**1** </td>   |  | | |


$C_5 = 3*3+4*2+5*1 = 22$

|             |    |    |    |    |    |    |    |    |      |  $C_5$   |
|--------------|----|----|----|----|----|----|----|----|------|----------|
| Input Array  |    |    | 1 | 2 <td style="border: 1px solid red;">**3** </td>  <td style="border: 1px solid red;">**4** </td> <td style="border: 1px solid red;">**5** </td>| | | |**22** |
| Kernel       |    |    |    |<td style="border: 1px solid red;">**3** </td>  <td style="border: 1px solid red;">**2** </td>   <td style="border: 1px solid red;">**1** </td>    | | |


$C_6 = 4*3+5*2 = 22$

|            |    |    |    |    |    |    |    |    |      |  $C_6$   |
|--------------|----|----|----|----|----|----|----|----|------|----------|
| Input Array  |    |    | 1 | 2 | 3  <td style="border: 1px solid red;">**4** </td> <td style="border: 1px solid red;">**5** </td>| | | |**22** |
| Kernel       |    |    |    |  |<td style="border: 1px solid red;">**3** </td>  <td style="border: 1px solid red;">**2** </td> | 1     | | |


$C_7 = 5*3 = 15$

|             |    |    |    |    |    |    |    |    |      |  $C_7$   |
|--------------|----|----|----|----|----|----|----|----|------|----------|
| Input Array  |    |    | 1 | 2 | 3 | 4 </td> <td style="border: 1px solid red;">**5** </td>| | | |**15** |
| Kernel       |    |    |    |  |   |<td style="border: 1px solid red;">**3** </td>  | 2 | 1     | |


Result of the Convoultion of the input array A = [1,2,3,4,5] and the kernel = [1,2,3] is: 

$C = (A * kernel) = [1,4,10,16,22,22,15] $

#### What is a Transposed Convolution?

In a transposed convolution, the kernel is used to upsample the input data by inserting zeros between the original data points, and then applying a standard convolution operation.

In [ ]:
# Let's confirm the above manual calculations using Numpy convolve function:

# Define the input array and kernel
A = np.array([1, 2, 3, 4, 5])
kernel = np.array([1, 2, 3])

# Perform the convolution using NumPy's convolve() function
output_array = np.convolve(A, kernel, 'full')

print(output_array) 

In [ ]:
# define a Sequential Model
model = keras.Sequential()

# Add layers to th Sequential model

# First, define the shape of the input data using the sample length and height
model.add(layers.Input(shape=(x_train.shape[1], x_train.shape[2])))

# Add layer to perform 1D convolution of the input data
# filters - number of outputs in the convolution
# kernel_size - size of the kernel for convolution operation. By default, kernel is initialized by drawing random samples from a uniform distribution
# strides - the distance between the starting points of two consecutive convolutions. The amount by which kernel is shifted when sliding across the input array
# padding = "same" - padding with zeros evenly to the left/right of the input such that output has the same width dimension as the input
# Output: 3D matrix with the following shape: 'Number of sequences' x 'Input Array length/strides' x 'filters'
model.add(layers.Conv1D(filters=32, kernel_size=7, padding="same", strides=2, activation="relu"))

# Randomly set 20% of the outputs from the previous layer to 0.
# This is used to help reduce the risk of overfitting to the training data.
model.add(layers.Dropout(rate=0.2))

# Add layer to perform 1D convolution of the input data. Layer similar to the one above, but it has only 16 outputs
model.add(layers.Conv1D(filters=16, kernel_size=7, padding="same", strides=2, activation="relu"))

# Add transposed convolutional layer
# filters - number of outputs in the convolution
# kernel_size - size of the kernel for convolution operation. By default, kernel is initialized by drawing random samples from a uniform distribution
# padding = "same" - padding with zeros evenly to the left/right of the input such that output has the same width dimension as the input
# strides - the distance between the starting points of two consecutive convolutions. The amount by which kernel is shifted when sliding across the input array
# Output: 3D matrix with the following shape: 'Number of sequences' x 'Input Array length * 2' x 'filters'
model.add(layers.Conv1DTranspose(filters=16, kernel_size=7, padding="same", strides=2, activation="relu"))

# Randomly set 20% of the outputs from the previous layer to 0.
# This is used to help reduce the risk of overfitting to the training data.
model.add(layers.Dropout(rate=0.2))

model.add(layers.Conv1DTranspose(filters=32, kernel_size=7, padding="same", strides=2, activation="relu"))
model.add(layers.Conv1DTranspose(filters=1, kernel_size=7, padding="same"))

In [ ]:
# Compile the model and siplay the summary
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model.summary()

## Train the Model

In [ ]:
history = model.fit(
    x_train,
    x_train,
    epochs=50,
    batch_size=128,
    validation_split=0.1,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min")
    ],
)

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.show()

## Detecting anomalies

In [ ]:
# Get train MAE loss.
x_train_pred = model.predict(x_train)
train_mae_loss = np.mean(np.abs(x_train_pred - x_train), axis=1)

plt.hist(train_mae_loss, bins=50)
plt.xlabel("Train MAE loss")
plt.ylabel("No of samples")
plt.show()

# Get reconstruction loss threshold.
threshold = np.max(train_mae_loss)
print("Reconstruction error threshold: ", threshold)

#### Compare recontruction

In [ ]:
# Checking how the first sequence is learnt
plt.plot(x_train[0])
plt.plot(x_train_pred[0])
plt.show()

#### Prepare test data

In [ ]:
df_test_value = (df_daily_jumpsup - training_mean) / training_std
fig, ax = plt.subplots()
df_test_value.plot(legend=False, ax=ax)
plt.show()

# Create sequences from test values.
x_test = create_sequences(df_test_value.values)
print("Test input shape: ", x_test.shape)

# Get test MAE loss.
x_test_pred = model.predict(x_test)
test_mae_loss = np.mean(np.abs(x_test_pred - x_test), axis=1)
test_mae_loss = test_mae_loss.reshape((-1))

plt.hist(test_mae_loss, bins=50)
plt.xlabel("test MAE loss")
plt.ylabel("No of samples")
plt.show()

# Detect all the samples which are anomalies.
anomalies = test_mae_loss > threshold
print("Number of anomaly samples: ", np.sum(anomalies))
print("Indices of anomaly samples: ", np.where(anomalies))

## Plot anomalies

In [ ]:
# data i is an anomaly if samples [(i - timesteps + 1) to (i)] are anomalies
anomalous_data_indices = []
for data_idx in range(TIME_STEPS - 1, len(df_test_value) - TIME_STEPS + 1):
    if np.all(anomalies[data_idx - TIME_STEPS + 1 : data_idx]):
        anomalous_data_indices.append(data_idx)

In [ ]:
df_subset = df_daily_jumpsup.iloc[anomalous_data_indices]
fig, ax = plt.subplots()
df_daily_jumpsup.plot(legend=False, ax=ax)
df_subset.plot(legend=False, ax=ax, color="r")
plt.show()